# Prompt Optimization with Evidently: Code Review Quality Classifier
This tutorial demonstrates how to use Evidently's new `PromptOptimizer` API for optimizing prompts for LLM judges. 
We'll walk through optimizing a prompt that classifies the quality of code reviews written for junior developers.

## What you'll learn:
- How to set up a dataset for LLM evaluation
- How to define an LLM judge with a prompt template
- How to run the prompt optimization loop
- How to retrieve and inspect the best performing prompt

In [ ]:
# If you haven't installed the required packages yet:
# !pip install evidently openai pandas

In [ ]:
import pandas as pd

from evidently import Dataset, DataDefinition, LLMClassification
from evidently.llm.templates import BinaryClassificationPromptTemplate
from evidently.descriptors import LLMEval
from evidently.llm.optimization import PromptOptimizer

In [ ]:
# Load your dataset
review_dataset = pd.read_csv("../datasets/code_review.csv")
review_dataset.head()

In [ ]:
# Define how Evidently should interpret your dataset
dd = DataDefinition(
    text_columns=["Generated review", "Expert comment"],
    categorical_columns=["Expert label"],
    llm=LLMClassification(input="Generated review", target="Expert label", reasoning="Expert comment")
)

In [ ]:
# Convert your pandas DataFrame into an Evidently Dataset
dataset = Dataset.from_pandas(review_dataset, data_definition=dd)

In [ ]:
# Define a prompt template and judge for classifying code review quality
criteria = '''A review is GOOD when it's actionable and constructive.
A review is BAD when it is non-actionable or overly critical.'''

feedback_quality = BinaryClassificationPromptTemplate(
    pre_messages=[("system", "You are evaluating the quality of code reviews given to junior developers.")],
    criteria=criteria,
    target_category="bad",
    non_target_category="good",
    uncertainty="unknown",
    include_reasoning=True,
)

judge = LLMEval(
    alias="Code Review Judge",
    provider="openai",
    model="gpt-4o-mini",
    column_name="Generated review",
    template=feedback_quality
)

In [ ]:
# Initialize the optimizer and run optimization using feedback strategy
optimizer = PromptOptimizer("code_review_example", strategy="feedback", verbose=True)
await optimizer.arun(executor=judge, scorer="accuracy", dataset=dataset, repetitions=5)
# for sync version:
# optimizer.run(judge, "accuracy")

In [ ]:
# Show the best-performing prompt template found by the optimizer
print(optimizer.best_prompt())

In [ ]:
# inspect optimizer statistics & logs
optimizer.print_stats()

### LLM Judge from scratch

You can also create LLM judges from scratch if you have labeled dataset.

In [ ]:
from evidently.llm.optimization import BlankLLMJudge

optimizer = PromptOptimizer("code_review_scratch_example", strategy="feedback", verbose=True)
await optimizer.arun(executor=BlankLLMJudge(), scorer="accuracy", dataset=dataset, repetitions=5)

In [ ]:
new_judge = optimizer.best_executor()
new_judge

In [ ]:
optimizer.print_stats()

You can save and load optimized template for later use

In [ ]:
from evidently.descriptors import LLMJudge
from evidently.llm.templates import BaseLLMPromptTemplate

new_judge.template.dump("my_template.yaml")
template = BaseLLMPromptTemplate.load("my_template.yaml")
my_judge = LLMJudge(provider="openai", model="gpt-4o-mini", template=template, input_column="Generated review")